In [ ]:
from langchain.chat_models import init_chat_model
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode, tools_condition
import os

class State(TypedDict):
    messages: Annotated[list, add_messages]
    
@tool
def get_currency_value(currency_code: str) -> float:
    '''
    Return the current value of a currency against USD.
    :param currency_code: ISO currency code (e.g., "USD", "INR", "EUR")
    :return: current value of the currency against USD
    '''
    return {
        "USD": 1.0,     # US Dollar
        "INR": 0.012,   # Indian Rupee → 1 INR ≈ 0.012 USD
        "EUR": 1.09,    # Euro → 1 EUR ≈ 1.09 USD
    }.get(currency_code.upper(), 0.0)
    
tools = [get_currency_value]
os.environ["OPENAI_API_KEY"] = ""  # <-- paste your real key here

# Initialize OpenAI GPT model
llm = init_chat_model(
    "gpt-3.5-turbo",
    model_provider="openai",
    temperature=0
)

llm_with_tools = llm.bind_tools(tools)   

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

builder = StateGraph(State)

builder.add_node(chatbot)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "chatbot")
builder.add_conditional_edges("chatbot", tools_condition)
builder.add_edge("tools", "chatbot")
graph = builder.compile()

from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png())) 
state = graph.invoke({"messages": [{"role": "user", "content": "Who is the pm of indian in one line?"}]})
print(state["messages"][-1].content)